In [1]:
#standard imports
import pandas as pd
import numpy as np

#API tools
import requests
import json
from pandas.io.json import json_normalize
from urllib.request import Request, urlopen
import json

#visuals
import matplotlib as plt
import seaborn as sns

#text tools
from collections import Counter #word frequency counts
import re #regexs
from ast import literal_eval #turn strings that look like lists into actual lists
import itertools #use to return combinations of words in topic strings 

#Natural Language Processing
import nltk
import lda #Latent Dirichlet Allocation (create topics)
import gensim
from gensim import corpora, models #for constructing document term matrix
#from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from nltk import stem
from nltk.corpus import stopwords

#clustering tools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import hdbscan

#magic
%matplotlib inline

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x) #otherwise we have scientific notation

## Access All Socrata Open Data Portal Meta-Data with Series of API Calls
Socrata's Discovery API allows a user to access ample metadata and usage data for datasets from Socrata portals. However, the API does have some limitations:
- The API itself is designed for queried searches, not returning all data about every Socrata open data portal. As such, you have to embrace some workarounds to truly access all available data.
- Without changing the 'limit' parameter, the API only returns 100 datasets for a given search parameter (the datasets returned *do seem to be sorted by total views*)

Additionally, trying to return more than 600 datasets at a time is very slow (on this machine).

** To work around these limitations, we will create a list of URLs for city/state open government Socrata open data portal. We will iterate through this list, returning the top 600 pages of data for each city's portal. Then we will convert each returned result to pandas-readable JSON, and convert that JSON into a dataframe. Then we will concatenate each dataframe. **

*Example API Call: http://api.us.socrata.com/api/catalog/v1?domains=data.seattle.gov&limit=600*

### First, get a list of all Socrata open data portals (for governments in the U.S.)
We will use an API call to get this list, which we will use for more API calls.

In [3]:
url_request = requests.get('http://api.us.socrata.com/api/catalog/v1/domains') 
#this is an API call of basically only domain URLs

url_json = url_request.json() #turn this requests.get object into a JSON-style dict
url_df = pd.io.json.json_normalize(url_json, record_path = 'results')

In [4]:
url_df.head(4)

,count,domain
0,4,2014bonds.cityofws.org
1,69,amopen.amo.on.ca
2,84,bchi.bigcitieshealth.org
3,3,bea.data.commerce.gov


In [5]:
urls = url_df.domain

** We have some .ca, .mx, etc. URLs that clearly aren't portals in the U.S. Let's remove them: **

In [6]:
city_urls = urls[urls.str.contains('^[a-zA-Z0-9\-\.]+\.(gov|org|com|us)$')] #use regex to match only certain URLs
city_urls = city_urls.reset_index(drop=True)
city_urls.tail()

/Users/nathanzencey1/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


213              www.forsythfutures.org
214    www.mdchildhungerpartnership.org
215            www.metrochicagodata.org
216                 www.opendatanyc.com
217                   www.sudandata.org
Name: domain, dtype: object

In [7]:
city_urls.head(10)

0          2014bonds.cityofws.org
1        bchi.bigcitieshealth.org
2           bea.data.commerce.gov
3           bis.data.commerce.gov
4    brigades.opendatanetwork.com
5         bythenumbers.sco.ca.gov
6        census.data.commerce.gov
7                chhs.data.ca.gov
8             chronicdata.cdc.gov
9         churned-data.awcnet.org
Name: domain, dtype: object

** As we can see, we have to manually datasets that the human eye can tell aren't related to U.S. cities or states. These include federal and non-profit open data. We could build a complex regex to do this, but there's only 220 or so datasets total. **

(We will KEEP data from state portals)

** CAUTION -- Must confirm these are the right index numbers, as the data returned by the API changes (obviously) **


In [8]:
city_urls[160:]

160                 midashboard.michigan.gov
161                          mydata.iadb.org
162                   nist.data.commerce.gov
163                   noaa.data.commerce.gov
164                   ntia.data.commerce.gov
165                   ntis.data.commerce.gov
166                       opencity.fcgov.com
167               opendata.ci.richmond.ca.us
168             opendata.cityofhenderson.com
169              opendata.cityofmesquite.com
170                         opendata.fcc.gov
171              opendata.howardcountymd.gov
172              opendata.lasvegasnevada.gov
173                   opendata.miamidade.gov
174                        opendata.utah.gov
175                 openpaymentsdata.cms.gov
176            outcomestat.baltimorecity.gov
177                  performance.adcogov.org
178              performance.austintexas.gov
179                   performance.carson.org
180              performance.chattanooga.gov
181          performance.ci.janesville.wi.us
182      p

In [9]:
#went through df with human eye line by line
clean_urls = city_urls.drop(city_urls.index[[1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 24, 25, 36, 40, 41, 47, 48, 52, 53, 54, 
                                             56, 57, 58, 66, 71, 79, 83, 84, 91, 94, 96, 97, 99, 103, 104, 108, 109, 
                                             110, 121, 128, 133, 140, 144, 146, 147, 148, 152, 153, 157, 159, 161, 
                                             162, 163, 164, 165, 170, 175, 198, 199, 200, 202, 203, 204, 208, 211, 
                                             213, 214, 217, 37, 39]]) #forgot two outlier datasets here

clean_urls = clean_urls[~clean_urls.str.contains('^[a-zA-Z0-9\-\.]+\.(cms.gov)$')] #make sure all CMS are gone

/Users/nathanzencey1/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [10]:
clean_urls[0:50]

0            2014bonds.cityofws.org
10               cip.cityofnovi.org
12       dashboard.alexandriava.gov
13             dashboard.hawaii.gov
14              dashboard.plano.gov
15               dashboard.slco.org
16                   data.acgov.org
17                data.albanyny.gov
18                data.auburnwa.gov
19             data.austintexas.gov
20           data.baltimorecity.gov
21                    data.brla.gov
22            data.burlingtonvt.gov
23             data.cambridgema.gov
26           data.cincinnati-oh.gov
27            data.cityofboston.gov
28           data.cityofchicago.org
29          data.cityofevanston.org
30       data.cityofgainesville.org
31                data.cityofgp.com
32           data.cityofmadison.com
33            data.cityofnewyork.us
34            data.cityoftacoma.org
35    data.cityofwestsacramento.org
38                data.colorado.gov
42        data.countyofriverside.us
43                    data.cstx.gov
44                      data

In [11]:
clean_urls = clean_urls.reset_index(drop=True)
clean_urls.index

RangeIndex(start=0, stop=148, step=1)

### Create a function to return JSON-formatted dicts of usage and meta-data for each URL's datasets.

In [12]:
#a function that takes a pd.Series of Socrata open data portal URLs and returns data for EACH in list of JSON-format dicts
def get_json(urls): #urls arg should be a Series of open data portal URLs
    json_dicts = []
    for url in urls:
        request = requests.get('http://api.us.socrata.com/api/catalog/v1?domains={}&limit=600'.format(url))
        raw_data = request.json()
        json_dicts.append(raw_data)
    return json_dicts

Get JSON Dicts: (** One Minute+ Run Time Alert **)

In [13]:
json_dicts = get_json(clean_urls)

In [14]:
len(json_dicts) #we have JSON data from 164 cities/states

148

This is a hulking list of JSON-style dictionaries. Let's see if pandas can handle making them into dfs:

In [15]:
def json_to_df(json_dicts):
    final_df = pd.DataFrame()
    for d in json_dicts:
        df = pd.io.json.json_normalize(d, record_path = 'results')
        final_df = final_df.append(df)
    return final_df

In [16]:
data = json_to_df(json_dicts)

In [17]:
data.head(5)

,classification,link,metadata,owner,permalink,preview_image_url,resource
0,"{'categories': [], 'tags': [], 'domain_categor...",https://2014bonds.cityofws.org/dataset/Bonds-P...,{'domain': '2014bonds.cityofws.org'},"{'id': '3xde-ipm7', 'display_name': 'mjmartin'}",https://2014bonds.cityofws.org/d/9csq-7i9t,NaN,"{'name': 'Bonds Project Dataset (Official)', '..."
1,"{'categories': [], 'tags': [], 'domain_tags': ...",https://2014bonds.cityofws.org/dataset/Bonds-A...,{'domain': '2014bonds.cityofws.org'},"{'id': 'd2yk-enzf', 'display_name': 'Christian...",https://2014bonds.cityofws.org/d/xwab-9myw,NaN,"{'nbe_fxf': None, 'description': '', 'type': '..."
2,"{'categories': [], 'tags': [], 'domain_tags': ...",https://2014bonds.cityofws.org/dataset/Wards-2...,{'domain': '2014bonds.cityofws.org'},"{'id': 'vkxb-3xjh', 'display_name': 'James Chu...",https://2014bonds.cityofws.org/d/963m-jiy8,https://2014bonds.cityofws.org/views/963m-jiy8...,"{'nbe_fxf': None, 'description': '', 'type': '..."
3,"{'categories': [], 'tags': [], 'domain_tags': ...",https://2014bonds.cityofws.org/dataset/Project...,{'domain': '2014bonds.cityofws.org'},"{'id': '3xde-ipm7', 'display_name': 'mjmartin'}",https://2014bonds.cityofws.org/d/hz5n-bkfx,NaN,"{'nbe_fxf': None, 'description': '', 'type': '..."
0,"{'categories': [], 'tags': [], 'domain_categor...",https://cip.cityofnovi.org/dataset/Capital-Pro...,{'domain': 'cip.cityofnovi.org'},"{'id': 'nc6z-mbfi', 'display_name': 'Jessica'}",https://cip.cityofnovi.org/d/7nqp-ya57,NaN,"{'name': 'Capital Project Dataset', 'id': '7nq..."


In [18]:
data.shape #we have data on 35,000+ datasets

(33494, 7)

**Ok, now we just need to unpack the dicts in these columns and clean up our df **.

In [19]:
data = data.reset_index(drop=True)
data.head(10)

,classification,link,metadata,owner,permalink,preview_image_url,resource
0,"{'categories': [], 'tags': [], 'domain_categor...",https://2014bonds.cityofws.org/dataset/Bonds-P...,{'domain': '2014bonds.cityofws.org'},"{'id': '3xde-ipm7', 'display_name': 'mjmartin'}",https://2014bonds.cityofws.org/d/9csq-7i9t,NaN,"{'name': 'Bonds Project Dataset (Official)', '..."
1,"{'categories': [], 'tags': [], 'domain_tags': ...",https://2014bonds.cityofws.org/dataset/Bonds-A...,{'domain': '2014bonds.cityofws.org'},"{'id': 'd2yk-enzf', 'display_name': 'Christian...",https://2014bonds.cityofws.org/d/xwab-9myw,NaN,"{'nbe_fxf': None, 'description': '', 'type': '..."
2,"{'categories': [], 'tags': [], 'domain_tags': ...",https://2014bonds.cityofws.org/dataset/Wards-2...,{'domain': '2014bonds.cityofws.org'},"{'id': 'vkxb-3xjh', 'display_name': 'James Chu...",https://2014bonds.cityofws.org/d/963m-jiy8,https://2014bonds.cityofws.org/views/963m-jiy8...,"{'nbe_fxf': None, 'description': '', 'type': '..."
3,"{'categories': [], 'tags': [], 'domain_tags': ...",https://2014bonds.cityofws.org/dataset/Project...,{'domain': '2014bonds.cityofws.org'},"{'id': '3xde-ipm7', 'display_name': 'mjmartin'}",https://2014bonds.cityofws.org/d/hz5n-bkfx,NaN,"{'nbe_fxf': None, 'description': '', 'type': '..."
4,"{'categories': [], 'tags': [], 'domain_categor...",https://cip.cityofnovi.org/dataset/Capital-Pro...,{'domain': 'cip.cityofnovi.org'},"{'id': 'nc6z-mbfi', 'display_name': 'Jessica'}",https://cip.cityofnovi.org/d/7nqp-ya57,NaN,"{'name': 'Capital Project Dataset', 'id': '7nq..."
5,"{'categories': [], 'tags': [], 'domain_tags': ...",https://cip.cityofnovi.org/dataset/Capital-Pro...,{'domain': 'cip.cityofnovi.org'},"{'id': 'vkxb-3xjh', 'display_name': 'James Chu...",https://cip.cityofnovi.org/d/kuue-gg3b,NaN,"{'nbe_fxf': None, 'description': '', 'type': '..."
6,"{'categories': [], 'tags': [], 'domain_categor...",https://cip.cityofnovi.org/dataset/Funding-Sou...,{'domain': 'cip.cityofnovi.org'},"{'id': 'trij-xrnq', 'display_name': 'Meredith ...",https://cip.cityofnovi.org/d/5h3i-jdcp,NaN,"{'name': 'Funding Sources (Pie Chart)', 'id': ..."
7,"{'categories': [], 'tags': [], 'domain_categor...",https://cip.cityofnovi.org/dataset/Funding-Sou...,{'domain': 'cip.cityofnovi.org'},"{'id': 'trij-xrnq', 'display_name': 'Meredith ...",https://cip.cityofnovi.org/d/uuui-htp8,NaN,"{'name': 'Funding Sources', 'id': 'uuui-htp8',..."
8,"{'categories': [], 'tags': [], 'domain_tags': ...",https://cip.cityofnovi.org/dataset/City-Bounda...,{'domain': 'cip.cityofnovi.org'},"{'id': 'trij-xrnq', 'display_name': 'Meredith ...",https://cip.cityofnovi.org/d/9fp5-7dsy,https://cip.cityofnovi.org/views/9fp5-7dsy/fil...,"{'description': '', 'type': 'map', 'download_c..."
9,"{'categories': [], 'tags': [], 'domain_tags': ...",https://cip.cityofnovi.org/dataset/Projects/cs...,{'domain': 'cip.cityofnovi.org'},"{'id': 'trij-xrnq', 'display_name': 'Meredith ...",https://cip.cityofnovi.org/d/cspu-rwqv,https://cip.cityofnovi.org/views/cspu-rwqv/fil...,"{'description': '', 'type': 'map', 'download_c..."


## ++++ Computationally Intensive!! ++++

In [20]:
#get each column's dict as separate df
classification = data.classification.apply(pd.Series)
metadata = data.metadata.apply(pd.Series)
resource = data.resource.apply(pd.Series)

#mash_data = pd.concat([data, classification, metadata, resource])

In [21]:
resource.head(10)

,attribution,columns_datatype,columns_description,columns_field_name,columns_name,createdAt,description,download_count,id,name,nbe_fxf,obe_fxf,page_views,parent_fxf,provenance,type,updatedAt,view_count
0,None,"[location, text, photo, text, text, text, text...","[, , , , , , , , , , , , , , , ]","[location, icon, project_image, category_id, l...","[Automated Geocoding, Icon, Project Image, Cat...",2015-02-12T19:14:36.000Z,,15.000,9csq-7i9t,Bonds Project Dataset (Official),nan,nan,"{'page_views_last_week': 8, 'page_views_last_m...",None,official,dataset,2017-08-11T19:53:36.000Z,"{'page_views_last_week': 8, 'page_views_last_m..."
1,None,"[text, text, text]","[, , ]","[icon, name, id]","[Icon, name, id]",2015-02-05T18:41:29.000Z,,7.000,xwab-9myw,Bonds Application - Reference Table,nan,nan,"{'page_views_total': 66, 'page_views_total_log...",None,official,dataset,2015-04-17T15:30:47.000Z,"{'page_views_total': 66, 'page_views_total_log..."
2,None,[],[],[],[],2015-02-09T20:48:39.000Z,,7.000,963m-jiy8,Wards 2011.shp,nan,nan,"{'page_views_total': 23, 'page_views_total_log...",None,official,map,2016-08-30T22:00:21.000Z,"{'page_views_total': 23, 'page_views_total_log..."
3,None,"[text, text, text, text, text, number, text, t...","[, , , , , , , , , , , , , ]","[city, location_id, project_id, project_name, ...","[City, Location Id, Project Id, Project Name, ...",2015-04-15T05:15:46.000Z,,3.000,hz5n-bkfx,Project Dataset (Staging),nan,nan,"{'page_views_total': 15, 'page_views_total_log...",None,official,dataset,2015-04-15T05:20:19.000Z,"{'page_views_total': 15, 'page_views_total_log..."
4,None,"[text, text, text, text, text, text, text, tex...","[, , , , , , , , , Please be sure to include ""...","[document_10, document_9, document_8, document...","[Document 10, Document 9, Document 8, Document...",2015-08-28T12:03:36.000Z,,25.000,7nqp-ya57,Capital Project Dataset,nan,nan,"{'page_views_last_week': 0, 'page_views_last_m...",None,official,dataset,2017-07-27T14:24:01.000Z,"{'page_views_last_week': 0, 'page_views_last_m..."
5,None,"[text, text, text]","[, , ]","[id, name, icon]","[id, name, Icon]",2015-08-28T12:01:45.000Z,,22.000,kuue-gg3b,Capital Project Application - Reference Table,nan,nan,"{'page_views_total': 63, 'page_views_total_log...",None,official,dataset,2015-10-02T23:07:35.000Z,"{'page_views_total': 63, 'page_views_total_log..."
6,None,"[text, money]","[, ]","[funding_source, budget]","[Funding Source, Budget]",2016-08-09T19:40:33.000Z,,32.000,5h3i-jdcp,Funding Sources (Pie Chart),nan,nan,"{'page_views_last_week': 0, 'page_views_last_m...","[7nqp-ya57, kf75-36tp]",official,chart,2017-07-27T14:23:39.000Z,"{'page_views_last_week': 0, 'page_views_last_m..."
7,None,"[text, money]","[, ]","[funding_source, budget]","[Funding Source, Budget]",2016-07-13T16:52:30.000Z,,21.000,uuui-htp8,Funding Sources,nan,nan,"{'page_views_last_week': 4, 'page_views_last_m...","[7nqp-ya57, kf75-36tp]",official,filter,2017-07-27T14:23:39.000Z,"{'page_views_last_week': 4, 'page_views_last_m..."
8,None,[],[],[],[],2016-10-07T23:19:08.000Z,,0.000,9fp5-7dsy,City Boundary,nan,nan,"{'page_views_total': 27, 'page_views_total_log...",None,official,map,2016-10-07T23:20:31.000Z,"{'page_views_total': 27, 'page_views_total_log..."
9,None,[],[],[],[],2016-10-07T23:22:05.000Z,,0.000,cspu-rwqv,Projects,nan,nan,"{'page_views_total': 20, 'page_views_total_log...",None,official,map,2016-10-07T23:25:10.000Z,"{'page_views_total': 20, 'page_views_total_log..."


In [22]:
#unpack views:
views = resource.view_count.apply(pd.Series)
views.head(10)

,page_views_last_month,page_views_last_month_log,page_views_last_week,page_views_last_week_log,page_views_total,page_views_total_log
0,34.000,5.129,8.000,3.170,464.000,8.861
1,1.000,1.000,0.000,0.000,66.000,6.066
2,0.000,0.000,0.000,0.000,23.000,4.585
3,0.000,0.000,0.000,0.000,15.000,4.000
4,4.000,2.322,0.000,0.000,393.000,8.622
5,0.000,0.000,0.000,0.000,63.000,6.000
6,6.000,2.807,0.000,0.000,62.000,5.977
7,10.000,3.459,4.000,2.322,61.000,5.954
8,4.000,2.322,1.000,1.000,27.000,4.807
9,1.000,1.000,0.000,0.000,20.000,4.392


Concatenate all these dfs into one:

In [23]:
data_mash = pd.concat([resource, classification, views, metadata], axis=1)
data_mash[15:25]

,attribution,columns_datatype,columns_description,columns_field_name,columns_name,createdAt,description,download_count,id,name,...,domain_tags,tags,page_views_last_month,page_views_last_month_log,page_views_last_week,page_views_last_week_log,page_views_total,page_views_total_log,domain,license
15,None,"[percent, number, number, calendar_date]","[, , , ]",[percent_of_valid_parking_meter_problem_servic...,[Percent of valid parking meter problem servic...,2016-02-22T20:36:45.000Z,,17.000,gnby-xhxz,Parking Complaints Bar Chart,...,[],[],17.000,4.170,1.000,1.000,274.000,8.103,dashboard.alexandriava.gov,NaN
16,None,"[calendar_date, number]","[, ]","[fiscal_year_originated, count]","[Fiscal Year Originated, Count]",2016-08-09T20:05:43.000Z,,10.000,mzr8-ks94,Home Buyer Loans Graph,...,[],[],18.000,4.248,1.000,1.000,272.000,8.093,dashboard.alexandriava.gov,NaN
17,None,"[number, number, percent, calendar_date]","[, , , ]","[potholes_filled, lane_miles_swept, percent_of...","[Potholes Filled, Lane Miles Repaved, Percent ...",2015-10-09T16:19:29.000Z,,7.000,5kha-4jsz,Potholes Filled by FY (Bar Chart),...,[],[],38.000,5.285,0.000,0.000,269.000,8.077,dashboard.alexandriava.gov,NaN
18,None,"[number, text]","[, ]","[number_of_units, fiscal_year]","[NUMBER OF UNITS, Fiscal Year]",2015-09-04T11:17:55.000Z,,5.000,ikzj-e4dt,Lane Miles Repaved,...,[],[],28.000,4.858,2.000,1.585,264.000,8.050,dashboard.alexandriava.gov,NaN
19,None,"[number, number, text]","[, , ]","[lipos_admissions_per_100k, nvmhi_admission_pe...","[LIPOS Admissions per 100K, NVMHI Admission pe...",2016-06-27T20:10:49.000Z,,5.000,e5v5-i25b,NVMHI Admissions,...,"[delete, dchs]",[],38.000,5.285,1.000,1.000,263.000,8.044,dashboard.alexandriava.gov,NaN
20,None,"[number, percent, calendar_date, number, number]","[, , , , ]","[number_of_property_owners_trained, percent_of...","[Number of property owners trained, Percent of...",2015-10-29T19:42:42.000Z,Office of Housing Data,11.000,9vuk-i6y2,Property Owners Trainined,...,[housing],[],30.000,4.954,0.000,0.000,246.000,7.948,dashboard.alexandriava.gov,NaN
21,None,"[percent, calendar_date, number, percent, number]","[, , , , ]","[diversion_rate, calendar_year, missed_collect...","[Diversion rate, Calendar Year, Missed Collect...",2015-09-04T13:38:06.000Z,,6.000,ayk9-7672,Tons Recycling Dropoff Centers,...,[],[],15.000,4.000,0.000,0.000,246.000,7.948,dashboard.alexandriava.gov,NaN
22,None,"[percent, text, text, percent, text, calendar_...","[, , , , , , , , , , , , ]","[problem, protected_class_group, action, prob_...","[Percent Problem, Protected Class Group, Actio...",2016-01-13T17:58:05.000Z,,17.000,mfwu-24pf,Fair Housing Sites Tested Per Year,...,"[fair housing, housing]",[],20.000,4.392,0.000,0.000,237.000,7.895,dashboard.alexandriava.gov,NaN
23,None,"[number, calendar_date]","[, ]","[count, fyear]","[Count, FYEAR]",2016-07-12T20:20:28.000Z,DCHS Rent Relief Data. Well Being and Safety f...,7.000,gr5b-bvxa,Number of Residents Assisted Through Rent Relief,...,"[well, rent relief, well-being and safety for ...",[],26.000,4.755,0.000,0.000,233.000,7.870,dashboard.alexandriava.gov,NaN
24,None,"[calendar_date, number, text]","[, , ]","[season, pts, outcome]","[Season, PTS, Outcome]",2015-07-24T18:52:32.000Z,"Lakers game outcomes and Kobe box scores, 1996...",3.000,22w3-jbsg,"Kobe average points in wins and losses, by season",...,[],[],31.000,5.000,0.000,0.000,226.000,7.827,dashboard.alexandriava.gov,NaN


In [24]:
data_mash.columns

Index(['attribution', 'columns_datatype', 'columns_description',
       'columns_field_name', 'columns_name', 'createdAt', 'description',
       'download_count', 'id', 'name', 'nbe_fxf', 'obe_fxf', 'page_views',
       'parent_fxf', 'provenance', 'type', 'updatedAt', 'view_count',
       'categories', 'domain_category', 'domain_metadata', 'domain_tags',
       'tags', 'page_views_last_month', 'page_views_last_month_log',
       'page_views_last_week', 'page_views_last_week_log', 'page_views_total',
       'page_views_total_log', 'domain', 'license'],
      dtype='object')

In [25]:
cols_to_keep = ['name', 'description', 'attribution', 'columns_field_name', 'columns_name', 'type', 'categories', 
                'domain_category','domain_tags', 'provenance', 'download_count','page_views_last_month',
                'page_views_last_week','page_views_total', 
                'page_views_total_log', 'domain']
cities = data_mash[cols_to_keep]
cities[25:35]

,name,description,attribution,columns_field_name,columns_name,type,categories,domain_category,domain_tags,provenance,download_count,page_views_last_month,page_views_last_week,page_views_total,page_views_total_log,domain
25,Resident Survey: Air Quality,For the relevant summary reports and technical...,None,"[of_total_weight, answergroupstep1_group, comp...","[Response Percent (weighted), Answer, Comparis...",chart,[finance],None,"[resident survey, city strategic plan]",official,4.000,60.000,4.000,220.000,7.788,dashboard.alexandriava.gov
26,Rental Unit Gain and Loss Graph,,None,"[fiscal_year, units_lost, new_units_added]","[Fiscal Year, Rental Units Lost, New Rental Un...",chart,[],NaN,"[assisted rental, housing]",official,14.000,6.000,0.000,217.000,7.768,dashboard.alexandriava.gov
27,Education Level- Filter,Worfkforce Development Center Education Ranges,None,"[fystartdate, count, geom, fiscalyear, fyendda...","[FYStartDate, count, geom, FiscalYear, FYEndDa...",chart,[],NaN,"[education level, increase employment, workfor...",official,5.000,22.000,0.000,216.000,7.762,dashboard.alexandriava.gov
28,"Issue types, as a percent of all types",,None,"[fiscal_year, percent_unsubstantiated, percent...","[Fiscal Year, Percent Unsubstantiated, Percent...",chart,[],None,[],official,5.000,31.000,0.000,214.000,7.748,dashboard.alexandriava.gov
29,Resident Survey: Ease of Travel by Bike,For the relevant summary reports and technical...,None,"[of_total_weight, answergroupstep1_group, comp...","[Response Percent (weighted), Answer, Comparis...",chart,[finance],None,"[resident survey, city strategic plan]",official,2.000,61.000,1.000,214.000,7.748,dashboard.alexandriava.gov
30,Pothole Service Requests by Fiscal Year,,None,"[fiscal_year, count]","[Fiscal Year, Count]",chart,[],NaN,[],official,2.000,34.000,2.000,209.000,7.714,dashboard.alexandriava.gov
31,Signal Services Requests Column Graph,,None,[percent_of_signals_receiving_preventative_mai...,[Percent of signals receiving preventative mai...,chart,[],NaN,[],official,1.000,30.000,2.000,207.000,7.700,dashboard.alexandriava.gov
32,Local GDP 2016,,None,"[number_of_accounts, business_license_class_na...","[Number of Accounts, Business License Class Na...",chart,[finance],NaN,"[local gross domestic product, city strategic ...",official,4.000,39.000,0.000,207.000,7.700,dashboard.alexandriava.gov
33,Number of Youth Counseled through the Workforc...,"Work Force Development Center, Youth Participants",None,"[fytarget, fyenddate, geom, count, fystartdate...","[FYTarget, FYEndDate, geom, count, FYStartDate...",chart,[politics],NaN,"[youth work experience, work experience, workf...",official,4.000,44.000,2.000,206.000,7.693,dashboard.alexandriava.gov
34,DCHS_BH_Opioid Use,Drug use report from Alaina,None,"[primary_opiates_category_combined, fiscal_yea...","[Opiates Category Combined, Fiscal Year, Perce...",chart,[],NaN,"[behavioral health, detox, dchs]",official,4.000,9.000,0.000,204.000,7.679,dashboard.alexandriava.gov


In [26]:
cities.shape

(33494, 16)

## EDA/Data Cleaning
We are going to cluster these datasets by content and use those clusters for quantitative analysis. However, we first need to see what we're dealing with and drop and NaN data for views. 

** See how many NaN view counts there are - we'll want to drop these **

In [27]:
NAs = cities[cities.page_views_total.isnull()] 
len(NAs.index)

0

In [28]:
type(cities.iloc[0, 10])

numpy.float64

Our page views column are floats, not lists (which could potentially be empty, as in other columns), so we're hopefully getting the true number of NaNs (0). This would square with what we expect - unless something went seriously wrong, Socrata's API will return view data for every dataset. However, not every city/state tags their datasets with categories/tags -- hence, some are missing.

In [29]:
type(cities.attribution[0])

NoneType

Looks like the "attribution" attribute has many NoneTypes. That's ok - we don't want to use attribution for clustering, as it's too specific - we want to be able to cluster similar datasets across cities, whereas including attribution - e.g. "Dallas Police Department" -- will just skew our clusters to clustering within a given city.

### Now we need to choose what attributes we'll use to create a text 'mash' from which we will use natural language processing and clustering tools. 

We need to choose carefully - as noted with attribution, the model we will build to do this is not a genius. It basically comes down to word appearance frequency is giant vectors, and how close these vectors are to each other in mathematical space. So, we want to include text attributes that won't skew how a dataset is classified.

## Create "Mash" from categories and tags:

** This actually takes a fair amount of cleaning **

In [30]:
mash_df = cities.copy()

#domain category is tricky because it contains both NaNs and NoneTypes. We'll replace both with a nothing str
mash_df['domain_category'] = mash_df['domain_category'].replace([None], '') 

#combine domain_tags and categories lists of strings columns - easy enough
mash_df['mash'] = mash_df.domain_tags + mash_df.categories 

#CONVERT lists of strings into actual string
mash_df['mash'] = mash_df['mash'].apply(lambda x: ','.join(map(str, x)))

#now add domain_category string to our mash column
mash_df['mash'] = mash_df.mash.str.cat(mash_df.domain_category, sep=' ') #this adds blank space if domain_category blank

#NOW our mash column is real-life words separated by commas and spaces. we need to split them
mash_df.mash = mash_df.mash.str.strip() #remove trailing whitespace
mash_df.mash = mash_df.mash.str.replace(' ', ",") #now replace all whitespaces with a comma to make splitting easier
mash_df.mash = mash_df.mash.str.lower() #we need strings to all be lowercase for vectorizing purposes later
mash_df.mash = mash_df.mash.str.split(',') #split big string on comma into actual words

In [31]:
mash_df.iloc[433].mash

['environmental', 'protection']

In [32]:
mash_df[430:435]

,name,description,attribution,columns_field_name,columns_name,type,categories,domain_category,domain_tags,provenance,download_count,page_views_last_month,page_views_last_week,page_views_total,page_views_total_log,domain,mash
430,Children Who Are Confirmed By Child Protective...,,http://datacenter.kidscount.org/,"[location, timeframe, dataformat, date_time, g...","[Location, TimeFrame, DataFormat, Date Time, G...",dataset,[],,[],official,23.000,0.000,0.000,304.000,8.253,dashboard.hawaii.gov,[]
431,AABD Client Age - June 2013,,None,"[pctn, n, characteristics]","[PctN, N, Characteristics]",chart,[],,[],official,14.000,31.000,0.000,304.000,8.253,dashboard.hawaii.gov,[]
432,Reading Proficiency Changes over Time Line Chart,,Socrata,"[year_text, year_date, reading_proficiency_mee...","[Year, Date Time, Meets, Exceeds, Approaches, ...",chart,[education],,[],official,21.000,24.000,1.000,302.000,8.243,dashboard.hawaii.gov,[education]
433,Wastewater Percentage Reused,,None,"[percentage_reused, wastewater_reused_mgd, tot...","[Percentage Reused, Wastewater Reused (MGD), T...",chart,[],Environmental Protection,[],official,26.000,30.000,6.000,302.000,8.243,dashboard.hawaii.gov,"[environmental, protection]"
434,TImeLIne of Class Size,This data reflects average class sizes from 20...,None,"[year_string, class_size, year]","[Year String, Class Size, Year]",chart,[],,[class size],official,22.000,4.000,0.000,301.000,8.238,dashboard.hawaii.gov,"[class, size]"


## Data Cleaning
- Drop type "filter", which essentially counts datasets twice
- Drop any blank "mash" value - can't analyze data we don't have
- Get counts of the data we deleted

In [33]:
#before drop
len(mash_df.index)

33494

In [34]:
cleaning_df = mash_df.copy()

#drop filtered views
cleaning_df = cleaning_df[cleaning_df.type != 'filter']

print("Filtered Views Dropped: {}".format(len(mash_df.index) - len(cleaning_df.index)))

Filtered Views Dropped: 6730


In [35]:
cleaning_df.provenance.value_counts() #check what's here; we want only official

official     25659
community     1105
Name: provenance, dtype: int64

In [36]:
cleaning_df.provenance.isnull().sum() #no NaNs

0

In [37]:
cleaning_df = cleaning_df[cleaning_df.provenance == 'official'] #make sure we have only gov datasets here

In [38]:
len(cleaning_df.index)

25659

In [39]:
cleaning_df.mash

0                                                       []
1                                                       []
2                                                       []
3                                                       []
4                                                       []
5                                                       []
6                                                       []
8                                                       []
9                                                       []
10                                                      []
11                                                      []
12                                                      []
13               [point, in, time, homelessness, ss, dchs]
14                                               [housing]
15                                        [transportation]
16                                                      []
17                                                      

In [40]:
cleaning_df.head()

,name,description,attribution,columns_field_name,columns_name,type,categories,domain_category,domain_tags,provenance,download_count,page_views_last_month,page_views_last_week,page_views_total,page_views_total_log,domain,mash
0,Bonds Project Dataset (Official),,None,"[location, icon, project_image, category_id, l...","[Automated Geocoding, Icon, Project Image, Cat...",dataset,[],,[],official,15.000,34.000,8.000,464.000,8.861,2014bonds.cityofws.org,[]
1,Bonds Application - Reference Table,,None,"[icon, name, id]","[Icon, name, id]",dataset,[],,[],official,7.000,1.000,0.000,66.000,6.066,2014bonds.cityofws.org,[]
2,Wards 2011.shp,,None,[],[],map,[],,[],official,7.000,0.000,0.000,23.000,4.585,2014bonds.cityofws.org,[]
3,Project Dataset (Staging),,None,"[city, location_id, project_id, project_name, ...","[City, Location Id, Project Id, Project Name, ...",dataset,[],,[],official,3.000,0.000,0.000,15.000,4.000,2014bonds.cityofws.org,[]
4,Capital Project Dataset,,None,"[document_10, document_9, document_8, document...","[Document 10, Document 9, Document 8, Document...",dataset,[],,[],official,25.000,4.000,0.000,393.000,8.622,cip.cityofnovi.org,[]


### Remove all blank lists:

In [41]:
df = cleaning_df.copy()
df = df[df.astype(str).mash != "['']"] #drops all blank lists
df = df.reset_index(drop=True)

In [42]:
big_mash = df.copy()
print("Number of Records in big_mash df: {}".format(len(big_mash.index)))

Number of Records in big_mash df: 21840


** Clean up description string and get it into a tokenized list that we can easily add to the existing mash list of tokens: **

In [43]:
big_mash.description = big_mash.description.str.replace(',', ' ') #replace natural commas with white space
big_mash.description = big_mash.description.str.replace('-', ' ') 
big_mash.description = big_mash.description.str.replace('.', ' ')
big_mash.description = big_mash.description.str.replace('&', ' ') 
big_mash.description = big_mash.description.str.replace(':', ' ')
big_mash.description = big_mash.description.str.split()

In [44]:
big_mash['big_mash'] = big_mash.mash + big_mash.description 

** Now we must reset our index or our later topic percentage comparisons won't match! **

In [45]:
big_mash = big_mash.reset_index(drop=True)
big_mash.head()

,name,description,attribution,columns_field_name,columns_name,type,categories,domain_category,domain_tags,provenance,download_count,page_views_last_month,page_views_last_week,page_views_total,page_views_total_log,domain,mash,big_mash
0,Homelessness PIT Transitional Age Youth,[],None,[location_on_the_night_of_the_count_total_pers...,"[Location on the night of the count, Total Per...",chart,[],,"[point in time, homelessness, ss, dchs]",official,4.000,46.000,1.000,319.000,8.322,dashboard.alexandriava.gov,"[point, in, time, homelessness, ss, dchs]","[point, in, time, homelessness, ss, dchs]"
1,Fair Housing Complaints,[],None,"[violations, percent_found_to_be_compliant, si...","[Number of complaints, Percent of sites found ...",chart,[],,[housing],official,21.000,31.000,0.000,278.000,8.124,dashboard.alexandriava.gov,[housing],[housing]
2,Parking Complaints Bar Chart,[],None,[percent_of_valid_parking_meter_problem_servic...,[Percent of valid parking meter problem servic...,chart,[transportation],,[],official,17.000,17.000,1.000,274.000,8.103,dashboard.alexandriava.gov,[transportation],[transportation]
3,NVMHI Admissions,[],None,"[lipos_admissions_per_100k, nvmhi_admission_pe...","[LIPOS Admissions per 100K, NVMHI Admission pe...",chart,[],,"[delete, dchs]",official,5.000,38.000,1.000,263.000,8.044,dashboard.alexandriava.gov,"[delete, dchs]","[delete, dchs]"
4,Property Owners Trainined,"[Office, of, Housing, Data]",None,"[number_of_property_owners_trained, percent_of...","[Number of property owners trained, Percent of...",chart,[],,[housing],official,11.000,30.000,0.000,246.000,7.948,dashboard.alexandriava.gov,[housing],"[housing, Office, of, Housing, Data]"


In [46]:
big_mash.big_mash[12] #this looks great; do have to remember to make it all lowercase

['youth',
 'work',
 'experience',
 'work',
 'experience',
 'workforce',
 'development',
 'center',
 'dchs',
 'politics',
 'Work',
 'Force',
 'Development',
 'Center',
 'Youth',
 'Participants']

## EDA on full dataset (minus filtered views)

In [47]:
print("Number of Missing domain_category tags: {}".format(big_mash.domain_category.isnull().sum()))

Number of Missing domain_category tags: 0


In [48]:
print("Empty Mash Rows Dropped: {}".format(len(cleaning_df.index) - len(big_mash.index)))

Empty Mash Rows Dropped: 3819


In [49]:
print("Clean DataFrame Length: {}".format(len(big_mash.index)))

Clean DataFrame Length: 21840


In [50]:
def median_mash_len(df):
    lengths = []
    for ls in df.big_mash:
        lengths.append(len(ls))
    return np.median(lengths)

In [51]:
print("Median Mash Length: {}".format(median_mash_len(big_mash)))

Median Mash Length: 23.0


# Export this clean dataframe to csv for replicability
- The results of Socrata API calls change (as one would expect) - we're usually getting a few more datasets. Even with a saved LDA Model, it won't align correctly to our dataframe unless it's the one with the exact "document" (row) alignment that we created the model from.

In [52]:
big_mash.to_csv('big_mash_df_8.16')

# NTLK Topic Analysis to find latent topics in "mash" categories
- Goal: use topic word "umbrellas" to count views/downloads by umbrella.
- Drawbacks: LDA assumes a document has multiple topics. This may be true of our mash - e.g. government and finance - but sometimes it might be just a single topic being arbitrarily split.

Resources: 
- https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html
- https://algobeans.com/2015/06/21/laymans-explanation-of-topic-modeling-with-lda-2/

## Load an LDA Model
- This is very computationally intensive. Fortunately we can save and import models.
- We pick the number of topics that we want the algorithm to find. This is a very important parameter.
- LDA Model fitting is an iterative process. The algorithm starts out by assigning every word to a temporary topic. Then, for *every* word, it updates the topics by calculating:
    - How prevalent is that word across topics? Topics with a high prevalance of the word in question get a higher weight for that word's assignment.
     - How prevalent are topics within a document? If one topic within a document is more prevalent, it gets a higher weight.
     - Based on these two criteria, LDA then updates a word's topic and document assignment.

** Therefore, the more iterations of LDA you can run, the more accurate it gets **

In [53]:
mash = big_mash.big_mash

stop_words = stopwords.words('english') #list of stop words

stop = list(stop_words) #copy ls since we're gonna mess with it
stop.append('&') #this ampersand is giving us fits
stop.extend(['', 'data', 'dataset', 'datasets', '//data', 'http', 
             'https', 'html', 'www', '//www', "=", "gov", "gov)", "(gov"]) #add words that don't specify given subject
stop.extend("•") #special char that showed up in earlier model iterations
stop.extend("–") #ditto as above
stop.extend(np.arange(101).astype(str)) #remove common numbers 0-100
stop.extend(np.arange(1980, 2025).astype(str)) #remove common years 1979-2024

texts = [] #blank list to append to

for ls in mash:
    lowers = [word.lower() for word in ls]
    stopped_tokens = [word for word in lowers if not word in stop]
    texts.append(stopped_tokens)

In [54]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [140]:
lda_50_fifty = gensim.models.ldamodel.LdaModel(corpus, num_topics=50, id2word = dictionary, 
                                         passes = 50)
#this takes an hour or so to run with 8GB of RAM

#import a model from memory.

In [141]:
model = lda_50_fifty

##### Topic Composition of Documents

In [142]:
corpus_lda = model[corpus] #this is just a wrapper; calculates on the fly when you call it

In [143]:
#view topic composition of documents in topic, composition tuples
for doc in corpus_lda[10:13]:
    print(doc) 

[(10, 0.11762958898007686), (17, 0.22333333333333349), (25, 0.42413156060795021), (34, 0.12056320181460291), (43, 0.064342315264037206)]
[(3, 0.11415018260670458), (6, 0.23050319141682402), (10, 0.22444444443093206), (35, 0.32867995930909982)]
[(2, 0.17429820160787324), (6, 0.180150884817874), (15, 0.060000000000000053), (20, 0.13703944684429745), (22, 0.092639290398971522), (33, 0.13508719194943991), (37, 0.068813352801572636), (42, 0.04373633746232497), (43, 0.060000000000000039)]


In [144]:
corpus_lda_list = list(corpus_lda) #bit of a run time here, as corpus_lda was just a wrapper; this calcs on the fly

#we're going to use this list later for a df

In [145]:
model.show_topics(num_topics=52, formatted = False)

[(0,
  [('gis', 0.053101416799317361),
   ('special', 0.051869215666611783),
   ('use', 0.036818458384491562),
   ('nursing', 0.032942476730674644),
   ('software', 0.029502552801497377),
   ('view', 0.02938583014343487),
   ('cdc', 0.028177026329258822),
   ('chicago', 0.026278237411059179),
   ('files', 0.02332381818425202),
   ('esri', 0.019336884950505433)]),
 (1,
  [('healthy', 0.027699567051271398),
   ('name', 0.024597015504312779),
   ('field', 0.024271329833696901),
   ('file', 0.02417942056019555),
   ('information', 0.022663999908261859),
   ('lead', 0.019343093674325372),
   ('names', 0.01904769196641861),
   ('treatment', 0.013423478705997832),
   ('nys', 0.012775084987347097),
   ('info', 0.012712295448407136)]),
 (2,
  [('housing', 0.13683663001056828),
   ('development', 0.1069655061620606),
   ('building', 0.055831454360123413),
   ('permits', 0.050606546631572651),
   ('dashboard', 0.045936908244525364),
   ('permit', 0.040918938770656228),
   ('construction', 0.02499

### Our corpus index numbers and df index numbers DO line up, as we can see below:
- corpus[] returns integer ids and frequency for each string
- dictionary.token2id shows us the id for each string so we can look'
- big_mash.big_mash[] shows that cell's strings in our df

In [123]:
corpus[500]

[(6, 1)]

In [124]:
dictionary.token2id

{'point': 0,
 'time': 1,
 'homelessness': 2,
 'ss': 3,
 'dchs': 4,
 'housing': 5,
 'transportation': 6,
 'delete': 7,
 'office': 8,
 'fair': 9,
 'well': 10,
 'rent': 11,
 'relief': 12,
 'well-being': 13,
 'safety': 14,
 'older': 15,
 'adults': 16,
 'resident': 17,
 'survey': 18,
 'city': 19,
 'strategic': 20,
 'plan': 21,
 'finance': 22,
 'relevant': 23,
 'summary': 24,
 'reports': 25,
 'technical': 26,
 'appendices': 27,
 '<a': 28,
 'href="https': 29,
 'alexandriava': 30,
 'gov/performance/default': 31,
 'aspx?id=89091">click': 32,
 'here</a>': 33,
 'assisted': 34,
 'rental': 35,
 'education': 36,
 'level': 37,
 'increase': 38,
 'employment': 39,
 'workforce': 40,
 'development': 41,
 'center': 42,
 'worfkforce': 43,
 'ranges': 44,
 'local': 45,
 'gross': 46,
 'domestic': 47,
 'product': 48,
 'youth': 49,
 'work': 50,
 'experience': 51,
 'politics': 52,
 'force': 53,
 'participants': 54,
 'behavioral': 55,
 'health': 56,
 'detox': 57,
 'drug': 58,
 'use': 59,
 'report': 60,
 'alaina':

In [125]:
big_mash.big_mash[500]

['transportation']

### Evaluating Topics Composition of Specific Dataset:

**Dataset with only one word to identify it:**

In [126]:
big_mash.big_mash[500] #random topic with one word

['transportation']

In [127]:
#cribbed from here: http://nbviewer.jupyter.org/gist/boskaiolo/cc3e1341f59bfbd02726 
for index, score in sorted(model[corpus[500]], key=lambda tup: -1*tup[1]): #500th document
    print("Score: {}\t Topic: {} \n".format(score, model.print_topic(index, 15))) #15 word topics

Score: 0.5096153846153851	 Topic: 0.161*"transportation" + 0.043*"infrastructure" + 0.026*"traffic" + 0.022*"street" + 0.022*"parking" + 0.020*"city" + 0.017*"streets" + 0.017*"transit" + 0.014*"class" + 0.013*"information" + 0.012*"bike" + 0.011*"routes" + 0.010*"point" + 0.010*"road" + 0.010*"pedestrian" 



**Dataset with many words to identify it**

In [128]:
big_mash.big_mash[700]

['alameda',
 'county',
 'assessor',
 'defaulted',
 'properties',
 'economy',
 'finance',
 'housing',
 'and',
 'development',
 'Defaulted',
 'Properties',
 'by',
 'Year/City/Zipcode',
 'as',
 'of',
 'Feb',
 '26',
 '2016']

In [129]:
for index, score in sorted(model[corpus[700]], key=lambda tup: -1*tup[1]): #600th document
    print("Score: {}\t Topic: {} \n".format(score, model.print_topic(index, 15))) #15 word topics

Score: 0.3585164835164834	 Topic: 0.084*"property" + 0.041*"cook" + 0.041*"tax" + 0.022*"real" + 0.020*"condition" + 0.019*"address" + 0.016*"properties" + 0.013*"sale" + 0.012*"determine" + 0.012*"executive" + 0.012*"information" + 0.011*"gis/maps" + 0.011*"article" + 0.010*"commercial" + 0.010*"potentially" 

Score: 0.16081553782441746	 Topic: 0.163*"development" + 0.130*"housing" + 0.049*"economic" + 0.048*"permits" + 0.047*"building" + 0.039*"permit" + 0.019*"construction" + 0.018*"infrastructure" + 0.018*"buildings" + 0.015*"issued" + 0.015*"city" + 0.014*"licensing" + 0.013*"department" + 0.013*"monitoring" + 0.010*"residential" 

Score: 0.1442307692307692	 Topic: 0.375*"county" + 0.099*"san" + 0.056*"rates" + 0.039*"population" + 0.024*"us" + 0.020*"demographics" + 0.019*"000" + 0.018*"current" + 0.018*"per" + 0.015*"government" + 0.014*"rate" + 0.010*"agency" + 0.009*"taxing" + 0.009*"alameda" + 0.008*"cells" 

Score: 0.1442307692307692	 Topic: 0.193*"health" + 0.040*"services"

**Very interesting (and promising, for our model's purposes):**

We can see that the top topics by affinity are a housing/building topic, and a finance topic. Our model is working appropriately here, given this dataset is about defaulted properties!

## Also To Note:
- Our topics are sparse in terms of probability/composition. As mentioned above, they look very good the human eye, and can be useful, but remember that they go on much longer than 10 words, and that your top 10 words only compose like 10-15% of the topic (very approximate)
- **A significant weakness** of this model is that all topics formed are the same size; there is no way to make "clusters" (topics) of varying density. But in real life, we know we have very specific topics that really only make sense with 3-4 words, whereas broad categories (public safety) can make sense with a ton of words
- Normally, another LDA weakness is that it is "bag of words"; it doesn't take into account the placement of words in a sentence. However, since we are mainly using tags ("public safety", "health", etc), that's not much of an issue here!
- This model allows for words to be re-used in topics. This is good for our purposes - e.g. "public safety" and "public records".

## Where are there errors and uncertainty?
**Again, not every dataset is tagged thoroughy, accurately, or appropriately.** 
  - Some cities/portals just give their datasets weird names or use stock descriptions for every single category of open data.
    
**Proper names obviously skew results somewhat; a proper name doesn't really tell us about the content of a dataset.**
  - However, it's just not feasible to remove every proper name as a stopword; at least not without extensive trial and error
  - This effect is mitigated by using topic composition for calculating the percentage of views that a topic gets. "Maryland" is in a community capital projects group, but a Maryland police department mash row will only match around .05 (very approximate)- so, the erroneous boost in views is neglible.
  
**The human brain interprets our clusters at the end. It's up to us to make up "category" or "topic" names. Two LDA topics may be very similar IRL topics (we can see this with taxes/public assistance - probably because that's such a prevalent IRL category).** 
  - Then again, all of this is labeled by humans. Back to our first point, there is always room for disagreement/debate in what "subject" a dataset is about, and how narrow to make subjects.

# Calculate Popularity of Each Topic Tag:
** We have 27k+ datasets, each with their own "mash". We also have 42 topics. Each mash is composed of X% of a handful of topics (usually 3-6). We are going to give each topic proportional credit for a given dataset's popularity. So, if topic 38 composes 30% of a dataset's mash, it get's 30% of that dataset's quantitative value.**
- However, since an LDA model is probabilistic, it by nature finds topics to be 5-10% of a "document" (our mash). These topics are very marginal at best. So if a topic doesn't account for at least 10% of a document, it doesn't get any points.
    
** We also need to devise a metric for a dataset's popularity. It will be a weighted combination of that dataset's views and downloads. **
- There's no way to make this statistically exact, really. We want to give downloads more credit than just combining their raw totals with views, as a download indicates a dataset is more useful and utilized (what if people are just viewing a dataset, but do little with it?).
    
*The takeaway from all this is that our final metrics will be best read as proportional, not absolute comparisons.* 
- We are losing some interpretability; we certainly won't be able to take a proposed dataset and predict the views/downloads it will get. However, we will be able to say that certain categories are more broadly popular than others. 

In [130]:
stats = big_mash.copy()
stats = stats.assign(topic_comp = corpus_lda_list)
stats.head() #the topic_comp column are actual Python lists

,name,description,attribution,columns_field_name,columns_name,type,categories,domain_category,domain_tags,provenance,download_count,page_views_last_month,page_views_last_week,page_views_total,page_views_total_log,domain,mash,big_mash,topic_comp
0,Homelessness PIT Transitional Age Youth,[],None,[location_on_the_night_of_the_count_total_pers...,"[Location on the night of the count, Total Per...",chart,[],,"[point in time, homelessness, ss, dchs]",official,4.000,46.000,1.000,319.000,8.322,dashboard.alexandriava.gov,"[point, in, time, homelessness, ss, dchs]","[point, in, time, homelessness, ss, dchs]","[(2, 0.836538461538)]"
1,Fair Housing Complaints,[],None,"[violations, percent_found_to_be_compliant, si...","[Number of complaints, Percent of sites found ...",chart,[],,[housing],official,21.000,31.000,0.000,278.000,8.124,dashboard.alexandriava.gov,[housing],[housing],"[(42, 0.509615384615)]"
2,Parking Complaints Bar Chart,[],None,[percent_of_valid_parking_meter_problem_servic...,[Percent of valid parking meter problem servic...,chart,[transportation],,[],official,17.000,17.000,1.000,274.000,8.103,dashboard.alexandriava.gov,[transportation],[transportation],"[(26, 0.509615384615)]"
3,NVMHI Admissions,[],None,"[lipos_admissions_per_100k, nvmhi_admission_pe...","[LIPOS Admissions per 100K, NVMHI Admission pe...",chart,[],,"[delete, dchs]",official,5.000,38.000,1.000,263.000,8.044,dashboard.alexandriava.gov,"[delete, dchs]","[delete, dchs]","[(2, 0.673076923077)]"
4,Property Owners Trainined,"[Office, of, Housing, Data]",None,"[number_of_property_owners_trained, percent_of...","[Number of property owners trained, Percent of...",chart,[],,[housing],official,11.000,30.000,0.000,246.000,7.948,dashboard.alexandriava.gov,[housing],"[housing, Office, of, Housing, Data]","[(32, 0.254807692308), (42, 0.504807692308)]"


## One way of scoring - only when a topic composes more than .2 of a doc does it get "credit" for its share of that doc's log views + downloads.

In [131]:
def calculate_topic_pop(df):
    results_dict = {}
    
    for row_num in df.index:
        for tup in df.topic_comp[row_num]:
            if not tup[0] in results_dict:
                if tup[1] >= 0.2:
                    results_dict[tup[0]] = (tup[1] * (df.iloc[row_num].download_count + 
                                             df.iloc[row_num].page_views_total_log))
                else:
                    pass
            if tup[0] in results_dict:
                if tup[1] >= 0.2:
                    results_dict[tup[0]] += (tup[1] * (df.iloc[row_num].download_count + 
                                             df.iloc[row_num].page_views_total_log))
    return results_dict

# Calculate Stats for Full 27k Row DF:

In [132]:
to_count = stats.copy()
to_count = to_count.fillna(0) #have to fill NaNs for downloads, log views etc with 0 or we get all NaNs

In [133]:
topic_stats = calculate_topic_pop(to_count)

In [134]:
#topic_stats #uncomment to view all stats in a dict

Pop this into a dataframe and add our topics' top 10 words:

In [135]:
topic_pop = pd.DataFrame.from_dict(topic_stats, orient='index')
topic_pop = topic_pop.rename(index=str, columns={0:"Adjusted_Popularity"}) #rename column

In [136]:
topic_tuple_lists = list(model.show_topics(num_topics=52, formatted=False))
topics_list = []

for tup in topic_tuple_lists:
    topics_list.append(tup[1])

In [137]:
topic_pop = topic_pop.assign(Topic = topics_list)
topic_pop = topic_pop.sort_values(by='Adjusted_Popularity', ascending=False)

topic_pop.head(10)

,Adjusted_Popularity,Topic
26,8619041.057,"[(health, 0.0471887849267), (managed, 0.037846..."
32,8096569.101,"[(new, 0.208424198756), (york, 0.0918320858581..."
24,693241.189,"[(information, 0.0585443643112), (ny, 0.046041..."
23,565534.700,"[(updated, 0.0294416802935), (may, 0.028259325..."
33,486689.388,"[(public, 0.0628570834751), (guide, 0.03057086..."
51,458767.381,"[(book, 0.047539776413), (oregon, 0.0393268254..."
42,365362.772,"[(politics, 0.122831017285), (survey, 0.104500..."
35,351533.092,"[(official, 0.0357707944733), (user, 0.0335685..."
37,319909.534,"[(click, 0.0325822044035), (""about"", 0.0313622..."
3,235964.084,"[(government, 0.15077812924), (finance, 0.1343..."


In [138]:
topics = topic_pop.copy()
topics[["topic1", "topic2", "topic3", "topic4", 
           "topic5", "topic6", "topic7", "topic8", "topic9", "topic10"]] = topics.Topic.apply(pd.Series)

In [139]:
topics.head(10)

,Adjusted_Popularity,Topic,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10
26,8619041.057,"[(health, 0.0471887849267), (managed, 0.037846...","(health, 0.0471887849267)","(managed, 0.037846386038)","(system, 0.0368795341426)","(medicaid, 0.0354640743854)","(library, 0.0314087876793)","(tx, 0.0295253634688)","(retirement, 0.0268333240265)","(closed, 0.0241140976379)","(well, 0.0180447699517)","(medicare, 0.0172276201421)"
32,8096569.101,"[(new, 0.208424198756), (york, 0.0918320858581...","(new, 0.208424198756)","(york, 0.0918320858581)","(chart, 0.056039191215)","(percentage, 0.0286629722253)","(assessment, 0.027961995093)","(measure, 0.0223964785561)","(community, 0.0206674740126)","(among, 0.0154909361905)","(standards, 0.0141397845429)","(orleans, 0.0102885455957)"
24,693241.189,"[(information, 0.0585443643112), (ny, 0.046041...","(information, 0.0585443643112)","(ny, 0.0460418232778)","(plans, 0.0445352005695)","(operations, 0.0270312405883)","(plan, 0.0265352906718)","(area, 0.023487370992)","(region, 0.0221998676515)","(job, 0.0221065820686)","(act, 0.0215092039994)","(centers, 0.0188572033771)"
23,565534.700,"[(updated, 0.0294416802935), (may, 0.028259325...","(updated, 0.0294416802935)","(may, 0.0282593255611)","(information, 0.0248223715925)","(january, 0.0241034985648)","(time, 0.0180556564904)","(month, 0.0176770644316)","(case, 0.0158224246465)","(related, 0.0152849528387)","(currently, 0.0122448818705)","(days, 0.0120515799401)"
33,486689.388,"[(public, 0.0628570834751), (guide, 0.03057086...","(public, 0.0628570834751)","(guide, 0.0305708694451)","(works, 0.0298482898426)","(safe, 0.026169243867)","(safety, 0.0243501271842)","(police, 0.0215007343778)","(master, 0.0203037737215)","(calls, 0.0202772220445)","(records, 0.0198274677493)","(address, 0.0189994110989)"
51,458767.381,"[(book, 0.047539776413), (oregon, 0.0393268254...","(book, 0.047539776413)","(oregon, 0.0393268254903)","(investigations, 0.0291643740165)","(application, 0.0282125265718)","(demographics, 0.0266436615181)","(infrastructure, 0.0206936577684)","(ldd, 0.0184282975995)","(technology, 0.01817106472)","(detroit, 0.0173631520062)","(mobile, 0.0169579366804)"
42,365362.772,"[(politics, 0.122831017285), (survey, 0.104500...","(politics, 0.122831017285)","(survey, 0.104500456279)","(results, 0.0537021520314)","(election, 0.050329421996)","(elections, 0.0390137951509)","(government, 0.0341860146337)","(conducted, 0.0219913214098)","(seattle, 0.0206006156731)","(wa, 0.017329657561)","(blank, 0.0135726308637)"
35,351533.092,"[(official, 0.0357707944733), (user, 0.0335685...","(official, 0.0357707944733)","(user, 0.0335685231763)","(page, 0.0297537701678)","(information, 0.0296647203264)","(police, 0.027751211045)","(department, 0.0262710264587)","(use, 0.0261422558015)","(orleans, 0.0182195933031)","(cities, 0.0160771958413)","(web, 0.015988097599)"
37,319909.534,"[(click, 0.0325822044035), (""about"", 0.0313622...","(click, 0.0325822044035)","(""about"", 0.0313622496471)","(nursing, 0.0274741362134)","(dallas, 0.0252397094855)","(investment, 0.0232637630606)","(non, 0.0223643951046)","(see, 0.0210992453935)","(public, 0.020639078968)","(citizens, 0.0186739871449)","(link, 0.0180218780111)"
3,235964.084,"[(government, 0.15077812924), (finance, 0.1343...","(government, 0.15077812924)","(finance, 0.134384152487)","(economy, 0.0597586644164)","(city, 0.0407053347986)","(strategic, 0.0399048447556)","(account, 0.03817781315)","(items, 0.0367681466965)","(plan, 0.0358174203537)","(open, 0.0333079012571)","(weekly, 0.0211059730465)"


In [112]:
topics.tail()

,Adjusted_Popularity,Topic,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10
15,25001.373,"[(demographics, 0.0681411067115), (counties, 0...","(demographics, 0.0681411067115)","(counties, 0.0588369851398)","(population, 0.0413694875455)","(zone, 0.024689683286)","(state, 0.0231976024051)","(community, 0.0204276855839)","(cause, 0.0197025982148)","(median, 0.0192376107726)","(income, 0.0183062046177)","(average, 0.018064056865)"
51,24468.709,"[(new, 0.12921479265), (york, 0.0673394418208)...","(new, 0.12921479265)","(york, 0.0673394418208)","(state, 0.0647811908886)","(ny, 0.0405594578594)","(historic, 0.0217378800769)","(medicaid, 0.0204588770514)","(definition, 0.018726864696)","(insurance, 0.0184556532356)","(federal, 0.0163733115876)","(healthcare, 0.0142365238833)"
24,21840.562,"[(finance, 0.0350713762631), (council, 0.02768...","(finance, 0.0350713762631)","(council, 0.0276827676398)","(city, 0.0255235663833)","(government, 0.0246298531296)","(open, 0.0242077102939)","(retirement, 0.0241150465071)","(spending, 0.0217081760906)","(pension, 0.0206441172611)","(economy, 0.0194132868497)","(program, 0.0177363637798)"
19,21550.081,"[(housing, 0.137321423148), (development, 0.11...","(housing, 0.137321423148)","(development, 0.118397977134)","(building, 0.0569678294518)","(permits, 0.0489493291418)","(permit, 0.0395789488796)","(buildings, 0.0252003138088)","(construction, 0.0215024487237)","(infrastructure, 0.0199250154601)","(city, 0.0174975129408)","(residential, 0.0151441222349)"
48,18492.741,"[(endorsed, 0.0341605655217), (standard, 0.032...","(endorsed, 0.0341605655217)","(standard, 0.0324382605093)","(crest, 0.0294937924688)","(api, 0.0228685684423)","(clear, 0.0202114366645)","(vermont, 0.0183195095505)","(limitations, 0.0177531228574)","(maps, 0.0176058346454)","(graphs, 0.0159463806151)","(el, 0.0156924910013)"


## "Winner Take All" Scoring Format:
- The topic that composes the greatest share of a document is the only topic to get points. It gets whatever proportion of that topics' log views and downloads that it composes.

**We'll write two quick functions here for replicability:**

In [113]:
def winner_take_all_pop(df):
    results_dict = {}
    df = df.fillna(0) #have to fill NaNs or you'll get wonky results

    for row_num in df.index:
        tup_list = df.topic_comp[row_num] #list of (topic, doc composition) tuples
        
        #neat little trick to return only the tuple w/highest index[1] value
        winner_tuple = max(tup_list, key=lambda item:item[1])  
            
        if not winner_tuple[0] in results_dict: #if not in dict, add it with its score
            results_dict[winner_tuple[0]] = (winner_tuple[1] * (df.iloc[row_num].download_count + 
                                             df.iloc[row_num].page_views_total_log))
        else:
            pass
        
        if winner_tuple[0] in results_dict: #if in dict, increment that key's value with score
            results_dict[winner_tuple[0]] += (winner_tuple[1] * (df.iloc[row_num].download_count + 
                                             df.iloc[row_num].page_views_total_log))
    
    return results_dict

In [114]:
winners_dict = winner_take_all_pop(to_count)

In [115]:
def dict_to_df(d):
    top_topics = pd.DataFrame.from_dict(d, orient='index')
    top_topics = top_topics.rename(index=str, columns={0:"Adjusted_Popularity"}) #rename column
    
    #get a list of our topic names to put into a df
    topic_tuple_lists = list(model.show_topics(num_topics=52, formatted=False))
    
    #iterate and append
    topics_list = []
    for tup in topic_tuple_lists:
        topics_list.append(tup[1])
        
    top_topics = top_topics.assign(Topic = topics_list)
    top_topics = top_topics.sort_values(by='Adjusted_Popularity', ascending=False)
    
    top_topics[["topic1", "topic2", "topic3", "topic4", 
           "topic5", "topic6", "topic7", "topic8", "topic9", "topic10"]] = top_topics.Topic.apply(pd.Series)
    
    return top_topics

In [116]:
rankings = dict_to_df(winners_dict)

ValueError: Length of values does not match length of index

In [ ]:
rankings

## Display titles of strongest document matches for each "cluster" (topic)